<a href="https://colab.research.google.com/github/vishesh711/Zillow-Home-Value-Prediction/blob/main/Multivariate_Time_Series_Forecasting_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!kaggle competitions download -c zillow-prize-1

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 398, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


#Multivariate Time Series Forecasting with LSTM
Neural networks like Long Short-Term Memory (LSTM) recurrent neural networks are able to almost seamlessly model problems with multiple input variables. This is a great benefit in time series forecasting, where classical linear methods can be difficult to adapt to multivariate or multiple input forecasting problems.

In [29]:
from datetime import datetime
import numpy as np
import numpy as numpy
import pandas as pd
import pylab
import calendar
from scipy import stats
import seaborn as sns
from sklearn import model_selection, preprocessing
from scipy.stats import kendalltau
from sklearn.metrics import mean_squared_error
import warnings
import matplotlib.pyplot as plt
import pandas
## Keras comes here
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, BatchNormalization
#from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
#rom keras.wrappers.scikit_learn import KerasRegressor
from keras import callbacks
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

In [30]:
print('Loading train, prop and sample data')
train = pd.read_csv("train_2016_v2.csv", parse_dates=["transactiondate"])
prop = pd.read_csv('properties_2016.csv')
sample = pd.read_csv('sample_submission.csv')

Loading train, prop and sample data


Label Encoder
LabelEncoder is a utility class to help normalize labels categorical values and to encode such that they contain only values between 0 and n_classes-1.

Here, we LabelEncode the properties dataset.

In [31]:
print('Fitting Label Encoder on properties')
for c in prop.columns:
    prop[c]=prop[c].fillna(-1)
    if prop[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(prop[c].values))
        prop[c] = lbl.transform(list(prop[c].values))

Fitting Label Encoder on properties


In [32]:
print('Creating training set:')
df_train = train.merge(prop, how='left', on='parcelid')

print('Creating df_test  :')
sample['parcelid'] = sample['ParcelId']

print("Merge Sample with property data :")
df_test = sample.merge(prop, on='parcelid', how='left')

Creating training set:
Creating df_test  :
Merge Sample with property data :


In [33]:
df_train["transactiondate"] = pd.to_datetime(df_train["transactiondate"])
df_train['transactiondate_quarter'] = df_train['transactiondate'].dt.quarter

basedate = pd.to_datetime('2015-11-15').toordinal()
df_train['cos_season'] = \
        ( (pd.to_datetime(df_train['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
          (2*np.pi/365.25) ).apply(np.cos)
df_train['sin_season'] = \
        ( (pd.to_datetime(df_train['transactiondate']).apply(lambda x: x.toordinal()-basedate)) * \
          (2*np.pi/365.25) ).apply(np.sin)

#test dataset
df_test["transactiondate"] = pd.to_datetime('2016-11-15')
df_test['transactiondate_quarter'] = df_test['transactiondate'].dt.quarter

df_test['cos_season'] = np.cos( (pd.to_datetime('2016-11-15').toordinal() - basedate) * \
                                    (2*np.pi/365.25) )
df_test['sin_season'] = np.sin( (pd.to_datetime('2016-11-15').toordinal() - basedate) * \
                                    (2*np.pi/365.25) )

df_train_x = df_train.drop(['logerror','parcelid', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode',
                             'fireplacecnt', 'fireplaceflag'],axis=1)
df_train = df_train.drop(['parcelid', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode',
                             'fireplacecnt', 'fireplaceflag'], axis=1)

train_columns = df_train_x.columns
df_test=df_test[train_columns]

##LSTM Data Preparation
The first step is to prepare properties dataset for LSTM.This involves framing the dataset as a supervised learning problem and normalizing the input variables. We will frame the supervised learning problem as predicting the Log Error for a particular parcel Id given other features and conditions at the prior time step.

We can transform the dataset using the series_to_supervised() function that is developed below;

In [34]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [35]:
Values = df_train.values
test_values = df_test.values
values = Values.astype('float32')
test_values = test_values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
scaled_test = scaler.fit_transform(test_values)
reframed = series_to_supervised(scaled, 1, 1)
reframed_test = series_to_supervised(scaled,1,1)
reframed.drop(reframed.columns[58:116], axis=1, inplace=True)
reframed_test.drop(reframed_test.columns[56:111],axis =1,inplace=True)

train_X, train_Y = reframed.iloc[:80000,:-1], reframed.iloc[:80000,-1]
valid_X, valid_Y = reframed.iloc[80000:,:-1], reframed.iloc[80000:,-1]

train_X = np.array(train_X)
train_Y = np.array(train_Y)

valid_X = np.array(valid_X)
valid_Y = np.array(valid_Y)
test_X = np.array(reframed_test)

In [36]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
valid_X = valid_X.reshape((valid_X.shape[0], 1, valid_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_Y.shape, valid_X.shape, valid_Y.shape,test_X.shape)

(956, 1, 57) (956,) (0, 1, 57) (0,) (956, 1, 59)


In [41]:
# model = Sequential()
# model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
# model.add(Dense(1))
# model.compile(loss='mae', optimizer='adam')
# # fit network
# model.fit(train_X, train_Y, epochs=50, batch_size=72, validation_data=(valid_X, valid_Y), verbose=2, shuffle=False)
# yhat = model.predict(valid_X)
# valid_X = valid_X.reshape((valid_X.shape[0], valid_X.shape[2]))

# rmse = np.sqrt(mean_squared_error(valid_Y, yhat))
# print('Test RMSE: %.3f' % rmse)

# Print the shapes of the datasets to ensure they are correct and not empty
print('Train X shape:', train_X.shape)
print('Train Y shape:', train_Y.shape)
print('Valid X shape:', valid_X.shape)
print('Valid Y shape:', valid_Y.shape)

# Ensure the content is not empty
print('First row of Train X:', train_X[0] if train_X.shape[0] > 0 else 'Empty')
print('First row of Train Y:', train_Y[0] if train_Y.shape[0] > 0 else 'Empty')
print('First row of Valid X:', valid_X[0] if valid_X.shape[0] > 0 else 'Empty')
print('First row of Valid Y:', valid_Y[0] if valid_Y.shape[0] > 0 else 'Empty')

Train X shape: (956, 1, 57)
Train Y shape: (956,)
Valid X shape: (0, 1, 57)
Valid Y shape: (0,)
First row of Train X: [[0.49185398 0.         0.         0.         0.25       0.30769232
  0.         0.         0.3181818  0.         0.         0.14987291
  0.14987291 0.         0.         0.         0.         0.29729462
  0.27272728 0.33333334 0.19703574 0.         0.         0.38352585
  0.8412552  0.00200633 0.         0.         0.         0.
  0.         0.9426229  0.29645538 0.06690589 0.         0.
  0.24286923 0.5833334  0.         0.6666667  0.         0.
  0.         0.         0.9747024  0.75       0.01924738 0.02633956
  0.         0.02744794 0.0170863  0.         0.         0.9914912
  0.         0.8261748  0.8789671 ]]
First row of Train Y: 0.4611325
First row of Valid X: Empty
First row of Valid Y: Empty


In [42]:
# Define the model
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# Fit the network
model.fit(train_X, train_Y, epochs=50, batch_size=72, validation_data=(valid_X, valid_Y), verbose=2, shuffle=False)

# Make predictions
yhat = model.predict(valid_X)
valid_X = valid_X.reshape((valid_X.shape[0], valid_X.shape[2]))

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(valid_Y, yhat))
print('Test RMSE: %.3f' % rmse)


Epoch 1/50


ValueError: Expected input data to be non-empty.

###Summary:
In this notebook, we have implemented Simple neural networks with 5 layers for prediction of LogError = (log(Zestimate)-log(salesprice)) using 2016 property dataset and its corresponding log error values provided by zillow for home value prediction in Python using Keras and tensorflow deep learning libraries.

Finally, we have predicted logerror values of 2016 and 2017 for the last quarter (from November to December) in the test dataset. Calculated RMSE for the Network built can be seen as 0.017 which is improved from that which we got by simple neural network. we can aslo infer that there is minimal error in the logerror gives us the better predictions.Further, the model can improvised by add more layers or changing the backpropagation parameters.

